# Brugia Viral Search

The goal of this analysis is to identify and hopefully assemble to some degree the viruses that are found present in the Brugia RNA-Seq data.

The first step is to run kraken on the data to identify likely viruses from the RefSeq database. Given it is limited to the RefSeq database, it is surely not comprehensive enough to be extrememly beneficial in de novo assembly, so LMAT is going to be run as well.

All data for this project can be found here: /scratch/at120/brugia-rnaseq/brugia_viral_search
Currently only working with this run: 2015-07-24_H55MKBCXX

The Brugia data have been filtered out via tophat which may or may not be suffice. We will have to see

## H55MKBCXX Run

In [ ]:
qsub run-kraken.sh

Once kraken has generated output, obtaining the reads that are viral and not bactieria, arcahea, or phiX can be done with the following command

In [ ]:
grep 'Virus' kraken.translate.out | grep -v 'phiX' | cut -f1 > viral-read-names.txt

Once these read names are generate the below script can be executed to extract the reads that are either: 
A) viral and not phiX based on the kraken classification
B) not listed in the kraken output since these are "non-Brugia/wolbachia" reads.

In [ ]:
python extract-viral-reads.py

Once that's done we need to pair up the reads into pairs and orphans

I need to add the functionality to original extraction script but for now I'm going to use the khmer script

In [ ]:
module load biopython
python extract-paired-reads.py all-unmapped.r1.viral.fastq all-unmapped.r2.viral.fastq all-unmapped.r1.viral.pe.fastq all-unmapped.r2.viral.pe.fastq all-unmapped.viral.se.fastq

For the first run, I'm sending it through IDBA to see if there's anything fruitful without manipulating the data (normalizing, partioning, more filtering, etc.).

In [ ]:
python interleave-fastq-to-fasta.py all-unmapped.r1.viral.pe.fastq all-unmapped.r2.viral.pe.fastq > all-unmapped.viral.interleaved.fasta

qsub -v path=/scratch/at120/brugia-rnaseq/brugia_viral_search,fasta=all-unmapped.viral.interleaved.fasta run-idba.sh

Putting this through IDBA, blasting, and then running Krona yielded 0 virus contigs LOL.

### Choristoneura occidentalis granulovirus

This, next to PhiX, is the most prevalent virus in this run, so I'm going to start with this one first.

http://www.ncbi.nlm.nih.gov/nuccore/NC_008168.1

#### Mapping

I'm mapping the reads to the genoem initially to see what kind of overage we're getting and if a near complete genome is possible.

I think the insert size is -50? Bowtie2 doesn't allow negative insert sizes so that sucks

In [ ]:
module load bowtie2/2.2.7
module load samtools/intel/1.3
cd /scratch/at120/brugia-rnaseq/brugia_viral_search/suspected-viruses/Choristoneura_occidentalis_granulovirus

bowtie2-build choristoneura_genome.fasta choristoneura_genome.fasta

bowtie2 \
-p 12 \
--very-sensitive-local \
--un-conc non-choco-virus.fastq \
-x choristoneura_genome.fasta \
-1 all-unmapped.r1.viral.pe.fastq \
-2 all-unmapped.r2.viral.pe.fastq \
-S choristoneura_genome.sam

module load samtools/intel/1.3
samtools view -b -o choristoneura_genome.bam choristoneura_genome.sam

4011418 reads; of these:
  4011418 (100.00%) were paired; of these:
    968151 (24.13%) aligned concordantly 0 times
    3041538 (75.82%) aligned concordantly exactly 1 time
    1729 (0.04%) aligned concordantly >1 times
    ----
    968151 pairs aligned concordantly 0 times; of these:
      678294 (70.06%) aligned discordantly 1 time
    ----
    289857 pairs aligned 0 times concordantly or discordantly; of these:
      579714 mates make up the pairs; of these:
        579000 (99.88%) aligned 0 times
        580 (0.10%) aligned exactly 1 time
        134 (0.02%) aligned >1 times
92.78% overall alignment rate

The majority of these reads mapped to a very small segment that was presumably one of the unique regions chosen by Kraken, hence why the majority of these reads were assigned to this virus.

I'm concluding this isn't present in this sample.

![alt text](/Users/alan/projects/brugia_viral_search/choco-virus/igv_snapshot.png")

### Simbu Virus

http://www.ncbi.nlm.nih.gov/genome/?term=Simbu%20virus

all of the reference sequences can be found with the kraken database: /scratch/at120/shared/db/kraken/standard/library/Viruses/

This one had the same phenomenon, so rule this out.

### All Kraken Virsues

I'm going to map all of the reads to the list of viruses that have more than 500 reads assigned to them and see what comes of it. It'll be quicker this way to interrogate potentially full genomes.

In [ ]:
cd /scratch/at120/brugia-rnaseq/brugia_viral_search/all-kraken-viruses

cat /scratch/at120/shared/db/kraken/standard/library/Viruses/*/*fna > all-kraken-seqs.fasta

bowtie2-build all-kraken-seqs.fasta all-kraken-seqs.fasta
bowtie2 \
-p 12 \
--very-sensitive-local \
--un-conc unconc.fastq \
-x all-kraken-seqs.fasta \
-1 all-unmapped.r1.viral.pe.fastq \
-2 all-unmapped.r2.viral.pe.fastq \
-S all-kraken-seqs.sam

samtools view -b -o all-kraken-seqs.bam  all-kraken-seqs.sam
samtools sort -o all-kraken-seqs.sort.bam all-kraken-seqs.bam
samtools index all-kraken-seqs.bam

The top 15 of these all had a 100bp region where a lot of reads mapped, nothing fruitful other than that.

### LMAT

Since kraken failed pretty miserably in providing any decent results, I'm going to give Kraken a try to see if that does any better at providing me a frame of reference for finding viruses.

## All Brugia Runs
Meanwhile, I merged all the brugia sequencing runs, ran tophat and kraken and same process as before

In [ ]:
cd /scratch/at120/brugia-rnaseq/brugia_viral_search/all-runs

cat /data/cgsb/gencore/out/Ghedin/2014-08-12_HAAULADXX/*R1* > all-r1.fastq.gz
cat /data/cgsb/gencore/out/Ghedin/2014-08-12_HAAULADXX/*R2* > all-r2.fastq.gz
cat /data/cgsb/gencore/out/Ghedin/2015-03-24_H2V3NBCXX/*/*n01* >> all-r1.fastq.gz
cat /data/cgsb/gencore/out/Ghedin/2015-03-24_H2V3NBCXX/*/*n02* >> all-r2.fastq.gz
cat /data/cgsb/gencore/out/Ghedin/2015-10-06_H5KNLBGXX/combined/*n01* >> all-r1.fastq.gz
cat /data/cgsb/gencore/out/Ghedin/2015-10-06_H5KNLBGXX/combined/*n02* >> all-r2.fastq.gz

qsub -v path=/scratch/at120/brugia-rnaseq/brugia_viral_search,fastq=all ../run-tophat.sh

This took WAY too long so I used the filter script seen here: